In [1]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [2]:
# import FunctionalCollections
import Iterators
import Pipe
import Compat
import JLD
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [3]:
@everywhere fold_indexes=load("brown_glove_folds.jld","fold_indexes")

@everywhere function fold_split(fold_ii, raw_bow_res)
    ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
    reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]
    
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[test_indexes]
    test_ground = ground_sents[test_indexes]
    training_sents = ground_sents[training_indexes]
    test_unordered_sents,test_ground, training_sents
end
    

In [4]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

import PyCall
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}})
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        kn_prob_dist[:prob]((given1, given2, event))
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: Use

In [ ]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
@everywhere const lm::Function = train_language_model(corpus)

In [ ]:
gc()
test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a","fine","comedy", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
#test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  shuffle(["no", "way", "."])
#test_bag =  shuffle(["no", "."])
#@time best_order(test_bag, lm, mem_limit=1000)

In [ ]:
using JuMP
using CoinOptServices
using AmplNLWriter
tic()
m=Model(solver=BonminNLSolver())
number_of_constraints=0
_lm = Dict{Tuple{S,S,S},Float64}()
function cached_lm(w1,w2,w3)
    get!(_lm, (w1,w2,w3)) do
       lm(w1,w2,w3)
    end
end


unordered_words  = test_bag
unordered_markers = [START_MARKER1; START_MARKER2; END_MARKER1; unordered_words...]
#Note that this lacks END_MARKER2

nodes = State{Int}[] #Named by word index

node_indexes_for_1st = Dict{Int, Vector{Int}}()
node_indexes_for_2nd = Dict{Int, Vector{Int}}()

function add_node!(ii,jj)
    push!(nodes, (ii,jj))

    node_indexes_for_i_1st = get!(()->Int[], node_indexes_for_1st, ii)
    push!(node_indexes_for_i_1st, length(nodes))

    node_indexes_for_j_2nd = get!(()->Int[], node_indexes_for_2nd, jj) 
    push!(node_indexes_for_j_2nd, length(nodes))
    #println("node:$(length(nodes)) |  $(unordered_markers[ii])($ii), $(unordered_markers[jj])($jj)")
end

add_node!(1, 2) #That is START_MARKER1-> STARTMARKER2

for ii in 1:length(unordered_markers)
    wi = unordered_markers[ii]
    if wi==END_MARKER1 || wi==START_MARKER1 continue end 
        #No node for the transition from END_MARKER1 to anything (not even END_MARKER2 as we do not have to model that as it is a constant prob 1)
        #No transition for START_MARKER1 to anyting but start maker 2
    for jj in [1:length(unordered_markers)]
        if ii==jj continue end
        wj = unordered_markers[jj]
        if wj==START_MARKER1 || wj==START_MARKER2 continue end
        add_node!(ii,jj)
    end
end

@defVar(m, x[1:length(nodes), 1:length(nodes)], Bin)

for class_index in 1:length(unordered_markers)
    w_class =  unordered_markers[class_index]
    if !(w_class==START_MARKER1 || w_class==START_MARKER2)#Not rquired to have either START_MARKER ever occur in second position
        @addConstraint(m, sum{x[ii,jj],ii=1:length(nodes), jj=node_indexes_for_2nd[class_index]}==1)
    end
    
    if !(w_class==END_MARKER1)#Not rquired to have END_MARKER ever occur in fist position
        @addConstraint(m, sum{x[ii,jj], ii=node_indexes_for_1st[class_index],jj=1:length(nodes)}==1)
    end
end


trans_prob = spzeros(length(nodes), length(nodes))
for (from_node_index, from_node) in enumerate(nodes)
    w1 = unordered_markers[from_node[1]]
    w2 = unordered_markers[from_node[2]]
    #If what was in the second state element does not end up in the first state element then it is not allowed.
    can_transition_to = Set(get(node_indexes_for_1st, from_node[2], Int[]))
        #You can transition to any node which has your second element as its first element
    for (to_node_index, to_node) in enumerate(nodes)
        if to_node_index in can_transition_to
            @assert(from_node[2]==to_node[1])
            w3= unordered_markers[to_node[2]]
            tp = cached_lm(w1,w2,w3)
            if tp>0.0
                trans_prob[from_node_index, to_node_index] = tp
                continue
            else
                #Banned as prob zero transitions not allowed
                @addConstraint(m, x[from_node_index,to_node_index]==0)
            end
            
        else
            #It is not a legal transition
            @addConstraint(m, x[from_node_index,to_node_index]==0)
        end
    end
end

iis, jjs, tp_values = findnz(trans_prob)
@setNLObjective(m, Max, prod{max((x[i,j]-1)^2, trans_prob[i,j]), i=iis, j=jjs})
#Note: max((x[i,j]-1)^2, trans_prob[i,j]) is trick for: x[i,j] ? trans_prob[i,j] : 1
toc()#34 seconds
@printval length(unordered_markers)
@printval length(m.linconstr)
trans_prob


In [ ]:
solve(m)
println("Objective value: ", getObjectiveValue(m))
x_val = getValue(x)
println("x = ", x_val)
x_iis,x_jjs, _  = findnz(x_val)
zip(x_iis,x_jjs) |> collect

In [ ]:
function get_prod(x, iis, jjs)
    net = 1.0
    for i in 1:size(x,1) 
        for j in 1:size(x,2)
            println(i,",", j, " =", x[i,j], " ", trans_prob[i,j])
            net*=max((x[i,j]-1)^2, trans_prob[i,j])
        end
    end
    net
end
values = falses(size(x))
values[1,3] = 1
values[3,6] = 1
values[6,7] = 1
println("----------\n")
get_prod(values, iis,jjs)
